In [4]:
import pandas as pd
from IPython.display import display


billing_data = pd.read_csv("billing_data.csv")
customer_data = pd.read_csv("customer_master_data.csv")
campaign_data = pd.read_csv("offer_campaign_data.csv")
usage_data = pd.read_csv("usage_data.csv")
support_data = pd.read_csv("customer_support_data.csv")

print("billing_data")
display(billing_data.head())
print("customer_data")
display(customer_data.head())
print("campaign_data")
display(campaign_data.head())
print("usage_data")
display(usage_data.head())
print("customer_support_data")
display(support_data.head())


billing_data


,CustomerID,Month,Amount,PaymentMethod,PaymentTimeliness
0,CUST1000,2023-01,34.016189,Cash,Late
1,CUST1000,2023-02,62.511343,Card,On-time
2,CUST1000,2023-03,96.466466,Online,Late
3,CUST1000,2023-04,83.373531,Cash,On-time
4,CUST1000,2023-05,71.806847,NaN,On-time


customer_data


,CustomerID,CustomerType,JoinDate,TerminationDate,Age,Region,Segment
0,CUST1000,Prepaid,2023-07-29,NaN,55,Riyadh,Premium
1,CUST1001,Postpaid,2022-08-29,NaN,20,Makkah,Premium
2,CUST1002,Prepaid,2023-07-18,NaN,40,Dammam,Mass
3,CUST1003,Prepaid,2022-04-06,2024-02-06,54,Jeddah,Premium
4,CUST1004,Prepaid,2023-04-30,NaN,54,Riyadh,Youth


campaign_data


,CampaignID,StartDate,EndDate,OfferType,TargetSegment,RedemptionRate
0,CAMPAIGN1,2023-01-01,2023-01-30,Discount,Youth,0.69
1,CAMPAIGN2,2023-01-31,2023-03-01,Discount,Premium,0.71
2,CAMPAIGN3,2023-03-02,2023-03-31,Discount,Premium,0.47
3,CAMPAIGN4,2023-04-01,2023-04-30,Discount,Youth,0.75
4,CAMPAIGN5,2023-05-01,2023-05-30,Data Booster,Mass,0.15


usage_data


,CustomerID,Month,VoiceMins,DataGB,SMS,RoamingGB
0,CUST1000,2023-01,645,22.099445,97,0.0
1,CUST1000,2023-02,454,17.412184,16,0.0
2,CUST1000,2023-03,633,12.950064,1,0.0
3,CUST1000,2023-04,818,3.164940,25,0.0
4,CUST1000,2023-05,10,10.992345,58,0.0


customer_support_data


,CustomerID,ContactDate,IssueCategory,ResolutionTime
0,CUST1000,2023-09-19,Network,5
1,CUST1001,2023-08-13,Network,4
2,CUST1001,2023-05-11,Device,5
3,CUST1001,2023-08-03,Device,5
4,CUST1007,2023-04-21,Other,5


**STEP 1:  Problem Understanding & Assumptions**

In [6]:
customer_data['Churn'] = customer_data['TerminationDate'].notnull().astype(int)
print("If TerminationDate is present → churned, If TerminationDate is NaN → active")

If TerminationDate is present → churned, If TerminationDate is NaN → active


**Revenue Loss**
Monthly revenue loss  = sum of **Amount** billed for churned customers, after their last active month

In [9]:
#Step 1: Mark churned customers
customer_data['Churn'] = customer_data['TerminationDate'].notnull().astype(int)

#Step 2: Merge churn info into billing
billing = pd.merge(billing_data, customer_data[['CustomerID', 'TerminationDate', 'Churn']], on='CustomerID', how='left')

#Step 3: Convert date fields
billing['Month'] = pd.to_datetime(billing['Month'], format='%Y-%m')
billing['TerminationDate'] = pd.to_datetime(billing['TerminationDate'])

#Step 4: Mark post-churn billing rows
billing['PostChurnBilling'] = (billing['Churn'] == 1) & (billing['Month'] > billing['TerminationDate'])



#Total Revenue form Churned customers before they left
churned_revenue_total = billing[billing['Churn'] == 1]['Amount'].sum()


#Step 5: Churn rate (rough estimate)
total_customers = customer_data.shape[0]
total_churned = customer_data['Churn'].sum()
churn_rate = total_churned / total_customers * 100

#Displaying summary
print(f"📉 Total Customers: {total_customers}")
print(f"🚪 Churned Customers: {total_churned}")
print(f"📊 Churn Rate: {churn_rate:.2f}%")
print(f"💸 Total Revenue from Churned Customers: {churned_revenue_total:.2f}")


📉 Total Customers: 200
🚪 Churned Customers: 125
📊 Churn Rate: 62.50%
💸 Total Revenue from Churned Customers: 80768.42


| KPI                              | Description                                        |
| -------------------------------- | -------------------------------------------------- |
| **Total Churn Rate**             | % of customers who got churned                     |
| **Churn by Segment/Region/Type** | Breakdowns to spot patterns                        |
| **Revenue Lost Due to Churn**    | Total billing amount from churned customers        |
| **Avg Revenue per User (ARPU)**  | Monthly average revenue/customer                   |
| **Offer Redemption Rate**        | Campaign acceptance and correlation with retention |
| **Support Complaints vs Churn**  | Average complaint count of churned vs retained     |


**STEP 2: Data Modelling**

| Field              | Definition                                   |
| ----------------   | -------------------------------------------  |
| **Churn**          | 1 if **TerminationDate** present, else 0     |
| **MonthlyRevenue** | Sum of **Amount** per user per month         |
| **ARPU**           | Avg of **Amount** per user over active months|
| **SupportIssues**  | Count of rows in support per user            |
| **RevenueLost**    | Sum of revenue from churned users            |


**STEP 3: Visualization Development**

In [ ]:
app.py

**STEP 4:  Storytelling & Presentation**

📌 Actionable Recommendations to Improve Retention
1. Target At-Risk Segments and Regions
Churn is without any doubt higher in certain customer segments and regions. What we can do is to focus retention efforts—like loyalty offers or personalized communication—on these groups to make them feel valued and reduce churn.

2. Improve Support Experience for Frustrated Customers
Customers with more complaints and slower resolution times are more likely to leave. Ramping up support and proactively reaching out to those with repeated issues can make a big difference.

3. Double Down on What Works in Campaigns
Some segments respond better to certain offers. Use that data to run smarter, if we can have more tailored campaigns—no need to offer everything to everyone.
